findings from this are 
1. currently our chunk sizes are too big (or must be too small), so we need to pick an embedding model that has a larger context window
2. if we still have to chunk our documents (longest one is 8.5k tokens, even if it fits in embedding model's context, it might overwhelm the actual LLM)
    1. if you use langchain_text_splitters it'll do the splitting and tokenization by itself, and the returned chunks won't have messed up spacing and casing etc. you have to mess with the separator characters/strings though
    2. chunking documents might also be desirable because "python notebook nexus" is closer to documents entirely about nexus than documents only partially about python notebooks ("python notebook" returns the right one). bigger embedding model doesn't fix this
3. there's duplicate pages from the allpages page that are in the html folder and scrapedtext file (which means duplicate or near duplicate chunks)
4. we'll also want to do some testing with seeing what range of similarity scores are relevant for us, cuz a lot of the time there's only a few sentences of context that answers a question

In [42]:
from transformers import AutoTokenizer
import os
from bs4 import BeautifulSoup

In [149]:
def get_html(file, sep=" "):
    with open("./dataset/raw_html/" + file, "r") as f:
        soup = BeautifulSoup(f, "html.parser")
        text = soup.get_text(separator=sep, strip=True)
        # text = soup
        return text
    return "wtf"

In [44]:
files = os.listdir("./dataset/raw_html")
files

['(N_A)Network_VPN_OSX.html',
 'Accounts.html',
 'Accounts_Collaborator.html',
 'ActiveDirectory.html',
 'AddingUMIACSCertificateAuthority.html',
 'Adobe.html',
 'AlternativePickup.html',
 'ApplicationResource.html',
 'Apptainer.html',
 'Archives.html',
 'ATL_ConferenceRooms.html',
 'Automounter.html',
 'Backups.html',
 'BarracudaSpamFirewall.html',
 'BarracudaSpamFirewall_QuarantinePassthrough.html',
 'BarracudaSpamFirewall_Scoring.html',
 'BarracudaSpamFirewall_SearchingQuarantine.html',
 'Barracuda_Spam_Firewall.html',
 'Bash.html',
 'BashForWindows.html',
 'BashForWindows10.html',
 'BitLocker.html',
 'BitLocker_PersonalUse.html',
 'Branding.html',
 'Caffe.html',
 'CBCB.html',
 'CCompilers.html',
 'CDebuggers.html',
 'CIFS.html',
 'ClassAccounts.html',
 'ClassAccounts_Manage.html',
 'Clear_Cache.html',
 'CMake.html',
 'CML.html',
 'CollaboratorAccount.html',
 'CompromisedPasswordFiltering.html',
 'ComputationalResource.html',
 'Compute_DataLocality.html',
 'ConferenceRooms.html',
 '

In [45]:
get_html("Nexus.html") 

'Nexus - UMIACS Nexus From UMIACS Jump to navigation Jump to search The Nexus is the combined scheduler of resources in UMIACS.  The resource manager for Nexus is SLURM .  Resources are arranged into partitions where users are able to schedule computational jobs.  Users are arranged into a number of SLURM accounts based on faculty, lab, or center investments. Contents 1 Getting Started 1.1 Access 1.2 Jobs 1.2.1 Interactive 1.2.2 Batch 2 Partitions 3 Quality of Service (QoS) 3.1 Job QoS 3.2 Partition QoS 4 Storage 4.1 Home Directories 4.2 Scratch Directories 4.2.1 Network Scratch Directories 4.2.2 Local Scratch Directories 4.3 Faculty Allocations 4.4 Project Allocations 4.5 Datasets Getting Started All accounts in UMIACS are sponsored.  If you don\'t already have a UMIACS account, please see Accounts for information on getting one.  You need a full UMIACS account (not a collaborator account ) in order to access Nexus. Access Your access to submission nodes (alternatively called login no

In [46]:
chosenfiles = [file for file in (file if "Nexus" in file or "SLURM" in file else None for file in files) if file is not None] # ? 
chosenfiles

['Nexus.html',
 'Nexus_Accounts.html',
 'Nexus_Apptainer.html',
 'Nexus_CBCB.html',
 'Nexus_CLIP.html',
 'Nexus_CML.html',
 'Nexus_Gamma.html',
 'Nexus_GPUs.html',
 'Nexus_MBRC.html',
 'Nexus_MC2.html',
 'Nexus_Tron.html',
 'Nexus_Vulcan.html',
 'Nexus_Vulcan_GPUs.html',
 'SLURM.html',
 'SLURM_ArrayJobs.html',
 'SLURM_ClusterStatus.html',
 'SLURM_JobStatus.html',
 'SLURM_JobSubmission.html',
 'SLURM_Preemption.html',
 'SLURM_Priority.html']

In [47]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
tokenizer.__dict__

{'_tokenizer': Tokenizer(version="1.0", truncation=TruncationParams(direction=Right, max_length=128, strategy=LongestFirst, stride=0), padding=PaddingParams(strategy=Fixed(128), direction=Right, pad_to_multiple_of=None, pad_id=0, pad_type_id=0, pad_token="[PAD]"), added_tokens=[{"id":0, "content":"[PAD]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":100, "content":"[UNK]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":101, "content":"[CLS]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":102, "content":"[SEP]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":103, "content":"[MASK]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}], normalizer=BertNormalizer(clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True), pre_tokenizer=

In [48]:
MAX_TOKENS = 2048 
OVERLAP = 100

# original model we were using has max context length 512, and was trained with only 128 token long sequences
def chunk_text_with_overlap(text, max_tokens=MAX_TOKENS, overlap=OVERLAP, tokenizer=tokenizer):
    """Chunks text within token limits with overlap for context retention."""
    tokens = tokenizer.tokenize(text)  # Tokenize the input text
    chunks = []

    # Create sliding window chunks with overlap
    for i in range(0, len(tokens), max_tokens - overlap):
        chunk_tokens = tokens[i:i + max_tokens]
        chunk = tokenizer.convert_tokens_to_string(chunk_tokens)
        chunks.append(chunk)

    return chunks

In [49]:
from langchain.embeddings import HuggingFaceEmbeddings

In [50]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [51]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document

In [52]:
vector_store = InMemoryVectorStore(embedding=embedding_model)

In [53]:
for file in chosenfiles:
    chunks = chunk_text_with_overlap(get_html(file))
    for chunk in chunks:
        # embedding = embedding_model.embed_query(chunk) # inmemoryvectorstore doesn't check if you set embeddings when adding documents/texts.
        # the redis one fortunately does
        vector_store.add_texts([chunk])

Token indices sequence length is longer than the specified maximum sequence length for this model (5242 > 512). Running this sequence through the model will result in indexing errors


In [54]:
len(vector_store.store.keys())

35

In [55]:
# the documents are >512 tokens long, so I think it's only looking at the first 512
vector_store.similarity_search(query="python notebook nexus", k=8)

[Document(id='7f7412ce-d1e8-45c3-97ef-4cbf39559e32', page_content="nexus / cml - umiacs nexus / cml from umiacs jump to navigation jump to search the compute nodes from cml ' s previous standalone cluster have folded into nexus as of the scheduled maintenance window for august 2023 ( thursday 08 / 17 / 2023, 5 - 8pm ). the nexus cluster already has a large pool of compute resources made possible through college - level funding for umiacs and csd faculty. details on common nodes already in the cluster ( tron partition ) can be found here. please contact staff with any questions or concerns. contents 1 usage 2 partitions 3 accounts 4 qos 5 storage 5. 1 home directories 5. 2 project directories 5. 3 scratch directories 5. 3. 1 network scratch directory 5. 3. 2 local scratch directories 5. 4 datasets 5. 5 models usage you can ssh to nexuscml. umiacs. umd. edu to log in to a submission host. if you store something in a local directory ( / tmp, / scratch0 ) on one of the two submission hosts

In [56]:
# we also see that it seems to focus a lot on the link title. (python notebook is only in the body of SLURM, the first result)
# adding nexus to the end of our query in the previous cell just gave us a bunch of nexus pages
vector_store.similarity_search(query="python notebook", k=8)

[Document(id='c5132a82-b07a-4623-84db-40162fc71757', page_content="slurm - umiacs slurm from umiacs jump to navigation jump to search contents 1 simple linux utility for resource management ( slurm ) 1. 1 documentation 1. 2 commands 1. 2. 1 srun 1. 2. 2 salloc 1. 2. 3 sbatch 1. 2. 4 squeue 1. 2. 5 scancel 1. 2. 6 sacct 1. 2. 7 sstat 1. 3 modules 1. 4 running jupyter notebook on a compute node 1. 4. 1 setting up your python virtual environment 1. 4. 2 running jupyter notebook 2 quick guide to translate pbs / torque to slurm simple linux utility for resource management ( slurm ) slurm is an open - source workload manager designed for linux clusters of all sizes. it provides three key functions. first, it allocates exclusive or non - exclusive access to resources ( computer nodes ) to users for some duration of time so they can perform work. second, it provides a framework for starting, executing, and monitoring work ( typically a parallel job ) on a set of allocated nodes. finally, it ar

In [57]:
len(vector_store.store)

35

with chunking modified to 128 tokens (and overlap adjusted)

In [58]:
vector_store = InMemoryVectorStore(embedding_model)
chunks = [chunk for f in chosenfiles for chunk in chunk_text_with_overlap(get_html(f), max_tokens=128, overlap=40)]
print(len(chunks))
vector_store.add_texts(chunks)

543


['f00b972f-0564-4d7a-95cf-72f3af65bfa3',
 '06b5772d-b5ec-45ca-b95b-939c2019a9f9',
 'a76d677f-c77a-41b7-88bc-c98df6f879be',
 'aaa57ca0-e8c5-4d10-86ae-94a61d0026b5',
 'd3fe756b-3981-494b-9d14-f40bc91bc6c3',
 'a51571a4-7af9-46e8-a7af-5f1b328f9467',
 'cd0c4011-409c-4656-99c6-183ceaa36d7c',
 'd4d2aa37-ec68-432e-8088-898f6d9c042e',
 '2afd21fa-5cdb-4c4a-a306-93165215e41b',
 '36cfa6d1-cfde-4b2b-8bfc-87b4db753179',
 '9c0b45dc-25f7-49f8-9057-f82a042effe6',
 '302fc279-bff2-4f94-9997-83440f039e99',
 '55e327b8-695d-4ff4-b9dd-ee755c4accce',
 'bb99a095-1103-4c0c-aec7-91407bf9ca7d',
 '430b9edf-145e-4490-ae5a-275f2216badd',
 'e64cf6e5-5f1d-4f10-af79-956ce3486932',
 '484de1f3-223c-4648-bdbe-58809848fb7e',
 '0455a596-8721-476f-bed2-1c731e16cd05',
 '5a83b9d6-5747-408b-b871-a4a6bc05caad',
 'fcbfbdb9-bb19-4bdd-a73a-8b6c51231922',
 'a4867f5b-e552-4191-b9b7-7c5744ca8d47',
 '8f412f51-db3e-4375-8cfc-dd8f95c2f7ca',
 '0ff11fcf-8156-4814-b111-e318c2f2f566',
 'f42dd7c5-38e0-419f-96f9-5b1539c96fe1',
 '34b411a0-aa63-

In [59]:
# 100 overlap seemed decent from my testing but 20 is too small, but having 100 overlap with 128 length is a lot of chunks
vector_store.similarity_search(query="python notebook nexus", k=8)

[Document(id='a0f20021-e91b-4627-8e42-c74d531d64a7', page_content='your username is username and that you are using the nexus cluster, have been assigned the nexusgroup submission nodes, and are assigned compute node tron00. umiacs. umd. edu : ssh - n - f - l localhost : 8888 : tron00. umiacs. umd. edu : 8889 username @ nexusgroup. umiacs. umd. edu you can then open a web browser and type in localhost : 8888 to access the notebook. notes : later versions of jupyter have token authentication enabled by default -'),
 Document(id='aa4260fc-b790-4ed5-9ea1-1b98a8972074', page_content='nexus - scratch / username / fs / nexus - containers / pytorch / pytorch _ 1. 13. 0 + cu117. sif bash you can now write / run your own pytorch python code interactively within the container or just make a python script that you can call directly from the apptainer exec command for batch processing. shared containers portable images called singularity image format or. sif files can be copied and shared. nexus m

way bigger model testing idk if it works at all

In [177]:
tokenizer_big = AutoTokenizer.from_pretrained("dunzhang/stella_en_400M_v5")
tokenizer_big.__dict__
# this isn't right at all here the size is 

{'_tokenizer': Tokenizer(version="1.0", truncation=TruncationParams(direction=Right, max_length=512, strategy=LongestFirst, stride=0), padding=PaddingParams(strategy=Fixed(512), direction=Right, pad_to_multiple_of=None, pad_id=0, pad_type_id=0, pad_token="[PAD]"), added_tokens=[{"id":0, "content":"[PAD]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":100, "content":"[UNK]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":101, "content":"[CLS]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":102, "content":"[SEP]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":103, "content":"[MASK]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}], normalizer=BertNormalizer(clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True), pre_tokenizer=

In [178]:
# 512 sequence length, so again truncated
embedding_model_big = HuggingFaceEmbeddings(model_name="dunzhang/stella_en_400M_v5", model_kwargs={"trust_remote_code":True})

Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [254]:
chunks = [chunk for f in chosenfiles for chunk in chunk_text_with_overlap(get_html(f), max_tokens=1024, overlap=200, tokenizer=tokenizer_big)]
# above ~1000 for this specific question makes the retrievals bad
chunks

["nexus - umiacs nexus from umiacs jump to navigation jump to search the nexus is the combined scheduler of resources in umiacs. the resource manager for nexus is slurm. resources are arranged into partitions where users are able to schedule computational jobs. users are arranged into a number of slurm accounts based on faculty, lab, or center investments. contents 1 getting started 1. 1 access 1. 2 jobs 1. 2. 1 interactive 1. 2. 2 batch 2 partitions 3 quality of service ( qos ) 3. 1 job qos 3. 2 partition qos 4 storage 4. 1 home directories 4. 2 scratch directories 4. 2. 1 network scratch directories 4. 2. 2 local scratch directories 4. 3 faculty allocations 4. 4 project allocations 4. 5 datasets getting started all accounts in umiacs are sponsored. if you don ' t already have a umiacs account, please see accounts for information on getting one. you need a full umiacs account ( not a collaborator account ) in order to access nexus. access your access to submission nodes ( alternativel

In [255]:
vector_store = InMemoryVectorStore(embedding_model_big)
vector_store.add_texts(chunks)
vector_store

In [256]:
vector_store.similarity_search(query="python notebook nexus", k=8)

[Document(id='242f79d4-30d6-403c-9fc5-e93049b7c789', page_content=". running jupyter notebook on a compute node the steps to run a jupyter notebook from a compute node are listed below. setting up your python virtual environment create a python virtual environment on the compute node you are assigned and activate it. next, install jupyter using pip by following the steps here. you may also use other environment management systems such as conda if desired. running jupyter notebook after you ' ve set up the python virtual environment, submit a job, activate the environment within the job, and run the following command on the compute node you are assigned : jupyter notebook - - no - browser - - port = 8889 - - ip = 0. 0. 0. 0 this will start running the notebook on port 8889. note : you must keep this shell window open to be able to connect. if the submission node for the cluster you are using is not accessible via the public internet, you must also be on a machine connected to the umiacs

In [257]:
model = embedding_model_big.client # this model lets you transform the question embedding to be more like a long answer's embedding
# no clue if it's better for us 
vector_store.similarity_search_by_vector(embedding=model.encode("python notebook nexus", prompt_name="s2p_query"), k=8)

[Document(id='242f79d4-30d6-403c-9fc5-e93049b7c789', page_content=". running jupyter notebook on a compute node the steps to run a jupyter notebook from a compute node are listed below. setting up your python virtual environment create a python virtual environment on the compute node you are assigned and activate it. next, install jupyter using pip by following the steps here. you may also use other environment management systems such as conda if desired. running jupyter notebook after you ' ve set up the python virtual environment, submit a job, activate the environment within the job, and run the following command on the compute node you are assigned : jupyter notebook - - no - browser - - port = 8889 - - ip = 0. 0. 0. 0 this will start running the notebook on port 8889. note : you must keep this shell window open to be able to connect. if the submission node for the cluster you are using is not accessible via the public internet, you must also be on a machine connected to the umiacs

In [258]:
vector_store.similarity_search(query="python notebook", k=8)

[Document(id='242f79d4-30d6-403c-9fc5-e93049b7c789', page_content=". running jupyter notebook on a compute node the steps to run a jupyter notebook from a compute node are listed below. setting up your python virtual environment create a python virtual environment on the compute node you are assigned and activate it. next, install jupyter using pip by following the steps here. you may also use other environment management systems such as conda if desired. running jupyter notebook after you ' ve set up the python virtual environment, submit a job, activate the environment within the job, and run the following command on the compute node you are assigned : jupyter notebook - - no - browser - - port = 8889 - - ip = 0. 0. 0. 0 this will start running the notebook on port 8889. note : you must keep this shell window open to be able to connect. if the submission node for the cluster you are using is not accessible via the public internet, you must also be on a machine connected to the umiacs

In [259]:
vector_store.store.keys().__len__()

67

In [260]:
text = "HELLO MY NAME IS BOB. https://google.com/HELP-ME. lowercaser ewrare aw"
tokens = tokenizer_big.tokenize(text)  # Tokenize the input text
print(tokens)
print(tokenizer_big.convert_tokens_to_string(tokens))
# chunks = []

# # Create sliding window chunks with overlap
# for i in range(0, len(tokens), max_tokens - overlap):
#     chunk_tokens = tokens[i:i + max_tokens]
#     chunk = tokenizer.convert_tokens_to_string(chunk_tokens)
#     chunks.append(chunk)
tokenizer_big.decode(tokenizer_big.encode("(https://google.com/)"), clean_up_tokenization_spaces=True, skip_special_tokens=True)

['hello', 'my', 'name', 'is', 'bob', '.', 'https', ':', '/', '/', 'google', '.', 'com', '/', 'help', '-', 'me', '.', 'lower', '##case', '##r', 'e', '##wr', '##are', 'aw']
hello my name is bob. https : / / google. com / help - me. lowercaser ewrare aw


'( https : / / google. com / )'

In [261]:
from langchain_text_splitters import CharacterTextSplitter

In [262]:
tokenizer_big.model_max_length = 512

In [263]:
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(tokenizer_big, chunk_size=512, chunk_overlap=100, separator="\n")

In [264]:
html = get_html("Nexus.html", sep="\n")
html

'Nexus - UMIACS\nNexus\nFrom UMIACS\nJump to navigation\nJump to search\nThe Nexus is the combined scheduler of resources in UMIACS.  The resource manager for Nexus is\nSLURM\n.  Resources are arranged into partitions where users are able to schedule computational jobs.  Users are arranged into a number of SLURM accounts based on faculty, lab, or center investments.\nContents\n1\nGetting Started\n1.1\nAccess\n1.2\nJobs\n1.2.1\nInteractive\n1.2.2\nBatch\n2\nPartitions\n3\nQuality of Service (QoS)\n3.1\nJob QoS\n3.2\nPartition QoS\n4\nStorage\n4.1\nHome Directories\n4.2\nScratch Directories\n4.2.1\nNetwork Scratch Directories\n4.2.2\nLocal Scratch Directories\n4.3\nFaculty Allocations\n4.4\nProject Allocations\n4.5\nDatasets\nGetting Started\nAll accounts in UMIACS are sponsored.  If you don\'t already have a UMIACS account, please see\nAccounts\nfor information on getting one.  You need a full UMIACS account (not a\ncollaborator account\n) in order to access Nexus.\nAccess\nYour access 

In [265]:
chunks = text_splitter.split_text(html) # ????????????????????????????????????????????????????????????????????
print(len(chunks))
for c in chunks:
    print(len(tokenizer_big.tokenize(c)))
# print(chunks)

17
286
347
373
387
352
432
446
433
411
420
409
385
338
377
376
373
110


In [266]:
lengths = []
for file in files:
    lengths.append(len(tokenizer_big.tokenize(get_html(file))))
lengths

[917,
 977,
 444,
 229,
 749,
 417,
 649,
 210,
 2931,
 317,
 1069,
 207,
 147,
 755,
 427,
 896,
 426,
 768,
 322,
 615,
 626,
 720,
 2283,
 216,
 756,
 196,
 427,
 202,
 145,
 3876,
 910,
 322,
 369,
 214,
 454,
 230,
 184,
 1049,
 166,
 1080,
 1083,
 1083,
 1083,
 1083,
 579,
 842,
 849,
 951,
 964,
 601,
 421,
 484,
 498,
 344,
 519,
 848,
 554,
 1318,
 567,
 2096,
 1076,
 439,
 1082,
 798,
 190,
 564,
 158,
 404,
 405,
 412,
 524,
 538,
 424,
 698,
 968,
 263,
 1064,
 1052,
 526,
 264,
 254,
 161,
 495,
 791,
 222,
 542,
 179,
 568,
 476,
 317,
 387,
 737,
 243,
 855,
 498,
 864,
 392,
 966,
 444,
 977,
 261,
 252,
 957,
 327,
 315,
 326,
 302,
 312,
 335,
 462,
 486,
 680,
 1111,
 301,
 149,
 252,
 1366,
 192,
 980,
 661,
 230,
 170,
 1308,
 195,
 1351,
 288,
 1680,
 1953,
 286,
 434,
 895,
 404,
 415,
 980,
 429,
 178,
 321,
 231,
 247,
 811,
 2258,
 478,
 952,
 572,
 292,
 155,
 249,
 1394,
 417,
 143,
 398,
 398,
 266,
 471,
 2331,
 586,
 397,
 541,
 541,
 371,
 730,
 899,
 47

In [267]:
print(max(lengths))
for i, l, in enumerate(lengths):
    if l >= max(lengths) - 100:
        print(files[i], l)

8479
Show_available_nodes.html 8479
SLURM_JobSubmission.html 8464


In [268]:
for i, l in enumerate(lengths):
    if l in lengths[i + 1:]:
        print(files[i])

Accounts.html
Accounts_Collaborator.html
Adobe.html
Archives.html
ATL_ConferenceRooms.html
Backups.html
BarracudaSpamFirewall_QuarantinePassthrough.html
Bash.html
BashForWindows10.html
ClassAccounts_Manage.html
CompromisedPasswordFiltering.html
ConferenceRooms_ATL3100A.html
ConferenceRooms_ATL3100C.html
ConferenceRooms_ATL3100D.html
ConferenceRooms_Recording.html
CoreServices.html
Creating_service_admin_user_on_OSX.html
EuclidCluster.html
FileTransferProtocol.html
Fixing_Stuck_Standing_Desk.html
HPC.html
Iribe.html
Iribe_ConferenceRooms_HuddleRoom.html
Iribe_Mailroom.html
Jekyll.html
Jira.html
LANDesk.html
LDAP.html
MacOSDisplayModes.html
Main_Page.html
MalwareRecovery.html
Malware_virus_removal.html
Mathematica.html
MathematicaActivation.html
MFA.html
Mpich1.csh.html
NAS.html
NASProjects.html
NautilusThumbnails.html
Network.html
Network_VPN.html
Network_VPN_Android.html
NFShomes.html
NightlyBackups.html
Perl.html
QuICS.html
RedHat.html
RHEL.html
Security.html
Services_Compute_UserSupp